In [1]:

import numpy as np
import scipy
from scipy.misc import imread, imsave, imresize
import skimage
from skimage import data
from matplotlib import pyplot as plt
from skimage.util.shape import view_as_blocks
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import feature
from skimage.transform import warp, AffineTransform
from skimage import io
%matplotlib inline
from PIL import Image

#************************************Function SECTION************************************   
def SUB_B(image, (M,N)):
    sub_Block = view_as_blocks(image, block_shape=(M,N))
    return sub_Block
#************************************Function SECTION************************************   

def myGrad_X(sub_Block,(max_row,max_col)):
    grad_sub_Block=np.empty([max_col,max_row]) 
    for row in range(0,max_row):
        for col in range(1,max_col-1):
                grad_sub_Block[row,col]=sub_Block[row,col-1]-sub_Block[row,col+1]  
        grad_sub_Block[row,0]= sub_Block[row,1]-sub_Block[row,0]
        grad_sub_Block[row,max_col-1]= sub_Block[row,max_col-1]-sub_Block[row,max_col-2]            
    return  grad_sub_Block   
# *********************************************************************************
def myGrad_Y(sub_Block,(max_row,max_col)):
    grad_sub_Block=np.empty([max_col,max_row]) 
    for col in range(0,max_col):
        for row in range(1,max_row-1):
            grad_sub_Block[col,row]=(sub_Block[row-1,col]-sub_Block[row+1,col])/2.0   
        grad_sub_Block[col,0]= sub_Block[1,col]-sub_Block[0,col]
        grad_sub_Block[col,max_row-1]= sub_Block[max_row-1,col]-sub_Block[max_row-2,col]
    return  grad_sub_Block   

# *********************************************************************************
def Concat_images(img_block,width,Height):
    return np.reshape(img_block,(width,Height))
# *********************************************************************************
def Preprocess(img):
    img =imread(img)
    img_gray=skimage.color.rgb2gray(img)
#     print 'shape : ', img_gray.shape  
    resized_img=skimage.transform.resize(img_gray,(256,256))
#     print 'resized_img : ', resized_img.shape
#     plt.imshow(resized_img, cmap='gray')
#     plt.show()
    return resized_img
# *********************************************************************************
def magnitude(iGx,iGy):
    iGx2 = np.power(iGx,2)
    iGy2 =  np.power(iGy,2) 
    m= np.sqrt(np.add(iGx2,iGy2))
    return m

# *********************************************************************************
def merge(mag_img,deg):   
    m = np.empty(mag_img.shape,dtype=object)
    for i in range (0,mag_img.shape[0]): 
        for j in range (0, mag_img.shape[1]):
            m[i,j]= (mag_img[i,j] , deg[i,j])
    return m

# *********************************************************************************

def Proc_Block(Sub_Blocks_Theta,Sub_Blocks_deg):

    
    Theta_cells=SUB_B(Sub_Blocks_Theta, (8,8))
    Mag_cells=SUB_B(Sub_Blocks_deg, (8,8))
    Histogram_Blocks= np.empty([0])
    for i in range(len(Theta_cells)):
        for j in  range(len(Theta_cells)):
            Histogram=ComputeCellHistogram(Theta_cells[i,j],Mag_cells[i,j])
            Histogram_Blocks = np.concatenate((Histogram_Blocks,Histogram),axis=0)            
    norm=np.linalg.norm(Histogram_Blocks)
    Block_descriptor= Histogram_Blocks/norm
    return Block_descriptor
# *********************************************************************************
def ComputeCellHistogram(Theta_cells,Mag_cells):
    deg_cells = np.rad2deg(Theta_cells).flatten()
    Hist,bin_edges=np.histogram(deg_cells,bins=9, weights=Mag_cells.flatten(), range= (-180,180))
    return Hist 

In [2]:

def simple_HOG(img):            
    row_index=16
    col_index=16
    plot_nr=1
    imGx=myGrad_X(img, (256,256))
    # plt.imshow(imGx, cmap='gray')
    # plt.show()
    imGy=myGrad_Y(img, (256,256))
    imGy=np.transpose(imGy)
    # plt.imshow(imGy, cmap='gray')
    # plt.show()
    mag_img=magnitude(imGx,imGy)
    # plt.imshow(mag_img, cmap='gray')
    # plt.show()
    Theta =np.arctan2(imGy,imGx)     
    deg = np.rad2deg(Theta)    
    Sub_Blocks_Theta=SUB_B(Theta, (16,16))
    Sub_Blocks_mag  =SUB_B(mag_img, (16,16))
    # # print Sub_Blocks , Sub_Blocks.shape
    hog_descriptor=np.empty([0])
    for i in range(len(Sub_Blocks_Theta)):
        for j in range(len(Sub_Blocks_Theta)):
            Block_descriptor=Proc_Block(Sub_Blocks_Theta[i,j],Sub_Blocks_mag[i,j])
            hog_descriptor= np.concatenate((hog_descriptor,Block_descriptor),axis=0)              
    return hog_descriptor

In [3]:
# img='lenna.png'
# img=Preprocess(img) 
# hog_descriptor=simple_HOG(img)
# print 'hog_descriptor : \n' ,  hog_descriptor 
# print 'hog_descriptor size' , hog_descriptor.shape


In [4]:
import os 

class KNNClassifier:
    
    def __EuclideanDist(self,featureX):
        
        self.nearest_neighbour = None
        self.min_distance = None
        
        for (featureY,category) in self.train_data:  
#              self.distance=np.linalg.norm(feature - feature_img)
                subt=np.subtract(featureX,featureY)
#                 print 'subt', subt
                pw=np.power(subt,2)
#                 print 'pw', pw
                sm=np.sum(pw)
#                 print 'sm', sm
                self.distance= np.sqrt(sm)
#                 print self.distance
            
                if self.min_distance == None or self.distance < self.min_distance:
                    self.min_distance = self.distance
#                     print self.min_distance
                    self.nearest_neighbour = (feature,category)
            
        if self.nearest_neighbour is not None:
#             print self.min_distance
            return self.nearest_neighbour[1]
       
        return None
    
    def classify(self,Feature_Img,classificationMethod="EuclideanDist"):
        if classificationMethod == "EuclideanDist":
            return self.__EuclideanDist(Feature_Img)
            
        return None   
    
    def __init__(self,train_Imagelist):
        self.train_data=train_Imagelist
        

def getAllImages(path):
    Imagelist=[]
    for Parent_root,Parent_dirs, Parent_files in os.walk(path):
#         print dirs
#         print root
        for dir in Parent_dirs:
            sub_path=os.path.join(Parent_root,dir)
#         print sub_path
            for root,dirs, files in os.walk(sub_path):
                    for file in files:
                        sub_path_img=os.path.join(sub_path,file)
#                         print sub_path_img
                        Imagelist= Imagelist + [(sub_path_img,dir)]
       
    return Imagelist

path='C:\Users\master\Assignments\Computer Vision\Ex_2_1\scene_categories' 
Imagelist=getAllImages(path)
cnt=len(Imagelist)
print cnt 
# print 'Imagelist : \n ',Imagelist

def extractFeatures(Imagelist):
    FeaturesList=[]
    for (img_path,category) in Imagelist:
        img=Preprocess(img_path)   
        hog_descriptor=simple_HOG(img)
        FeaturesList=FeaturesList + [(hog_descriptor,category)]
    return FeaturesList

FeaturesList=extractFeatures(Imagelist)
for i in FeaturesList[0][0]:
    print 'FeaturesList :' , i 

457


C:\ProgramData\Anaconda2\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\ProgramData\Anaconda2\lib\site-packages\skimage\util\shape.py:94: RuntimeWarning: Cannot provide views on a non-contiguous input array without copying.
  warn(RuntimeWarning("Cannot provide views on a non-contiguous input "
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:81: RuntimeWarning: invalid value encountered in divide


FeaturesList : 0.114530670583
FeaturesList : 0.0404213336626
FeaturesList : 0.509515177448
FeaturesList : 0.144516507447
FeaturesList : 0.113684393633
FeaturesList : 0.0
FeaturesList : 0.368101770099
FeaturesList : 0.0837345483919
FeaturesList : 0.0
FeaturesList : 0.0635615708645
FeaturesList : 0.050193758152
FeaturesList : 0.616405346909
FeaturesList : 0.396262928422
FeaturesList : 0.0326582162954
FeaturesList : 0.0114888935239
FeaturesList : 0.0419277969716
FeaturesList : 0.00544882942685
FeaturesList : 0.0242258747797
FeaturesList : 0.00900665882871
FeaturesList : 0.00195102302968
FeaturesList : 0.00851324910432
FeaturesList : 0.0176916941797
FeaturesList : 0.00212545298582
FeaturesList : 0.000317451048156
FeaturesList : 0.000492301551302
FeaturesList : 0.000921981864758
FeaturesList : 0.00415468501161
FeaturesList : 0.0333359263058
FeaturesList : 0.00288622395669
FeaturesList : 0.00197516736767
FeaturesList : 0.0145517527531
FeaturesList : 0.0180003578265
FeaturesList : 0.000609217

FeaturesList : 0.0
FeaturesList : 0.00161186150774
FeaturesList : 0.0341002116942
FeaturesList : 0.0431337896274
FeaturesList : 0.00864295388702
FeaturesList : 0.00500975776679
FeaturesList : 0.00976546191463
FeaturesList : 0.0101228163882
FeaturesList : 0.0014266324744
FeaturesList : 0.0021779766276
FeaturesList : 0.000336971037857
FeaturesList : 0.0210912414592
FeaturesList : 0.0169824826025
FeaturesList : 0.00767320106334
FeaturesList : 0.0628194058854
FeaturesList : 0.180740319775
FeaturesList : 0.0650736158703
FeaturesList : 0.025014514285
FeaturesList : 0.151326043542
FeaturesList : 0.00132100120834
FeaturesList : 0.00177218989394
FeaturesList : 0.108954892212
FeaturesList : 0.0
FeaturesList : 0.0
FeaturesList : 0.241171554144
FeaturesList : 0.72593875634
FeaturesList : 0.180878438058
FeaturesList : 0.0
FeaturesList : 0.0200255414624
FeaturesList : 0.0
FeaturesList : 0.0174542972977
FeaturesList : 0.0152025815317
FeaturesList : 0.00768570574998
FeaturesList : 0.0499890703688
Feat

FeaturesList : 0.0
FeaturesList : 0.00678262716143
FeaturesList : 0.517425206867
FeaturesList : 0.0
FeaturesList : 0.0
FeaturesList : 0.0
FeaturesList : 0.0
FeaturesList : 0.0
FeaturesList : 0.0
FeaturesList : 0.0
FeaturesList : 0.0237615207565
FeaturesList : 0.106129830803
FeaturesList : 0.00475327981506
FeaturesList : 0.000485794447883
FeaturesList : 0.0015686862447
FeaturesList : 0.0
FeaturesList : 0.000991647508924
FeaturesList : 0.000543105393311
FeaturesList : 0.0
FeaturesList : 0.0422264801342
FeaturesList : 0.345071511496
FeaturesList : 0.00330205725394
FeaturesList : 0.0
FeaturesList : 0.0639236842481
FeaturesList : 0.317912579272
FeaturesList : 0.053969997567
FeaturesList : 0.0164374117511
FeaturesList : 0.0149030766497
FeaturesList : 0.147216328453
FeaturesList : 0.108416028835
FeaturesList : 0.0826212959296
FeaturesList : 0.288729049885
FeaturesList : 0.0
FeaturesList : 0.0285816093148
FeaturesList : 0.0402757329394
FeaturesList : 0.585338641195
FeaturesList : 0.14911199348

...

FeaturesList : 0.0384793707067
FeaturesList : 0.0593814804676
FeaturesList : 0.074946400011
FeaturesList : 0.197481555525
FeaturesList : 0.120156014005
FeaturesList : 0.0764274405306
FeaturesList : 0.0848401039338
FeaturesList : 0.196644000377
FeaturesList : 0.0872541776688
FeaturesList : 0.068474946492
FeaturesList : 0.180469697568
FeaturesList : 0.257301612263
FeaturesList : 0.331274713745
FeaturesList : 0.116200503441
FeaturesList : 0.061763429446
FeaturesList : 0.0545784302892
FeaturesList : 0.154268242769
FeaturesList : 0.0841058233762
FeaturesList : 0.0176194728572
FeaturesList : 0.0442781267141
FeaturesList : 0.0388914711257
FeaturesList : 0.174160084266
FeaturesList : 0.13925927342
FeaturesList : 0.191804438025
FeaturesList : 0.244068784638
FeaturesList : 0.156759556882
FeaturesList : 0.110056913803
FeaturesList : 0.0513381402391
FeaturesList : 0.0175508211328
FeaturesList : 0.00946916163053
FeaturesList : 0.263122171423
FeaturesList : 0.162925300859
FeaturesList : 0.0554568143

FeaturesList : 0.0726336525663
FeaturesList : 0.0432825937743
FeaturesList : 0.0468200210806
FeaturesList : 0.0171242010763
FeaturesList : 0.156763760379
FeaturesList : 0.00443859716958
FeaturesList : 0.0694871401871
FeaturesList : 0.117670499978
FeaturesList : 0.165906581143
FeaturesList : 0.205734148738
FeaturesList : 0.0415292257368
FeaturesList : 0.10467386246
FeaturesList : 0.312995889371
FeaturesList : 0.121449726618
FeaturesList : 0.0186294535355
FeaturesList : 0.0185797141912
FeaturesList : 0.10520203749
FeaturesList : 0.158269485077
FeaturesList : 0.106591785604
FeaturesList : 0.0214063407919
FeaturesList : 0.138583432464
FeaturesList : 0.180075527251
FeaturesList : 0.0599835467807
FeaturesList : 0.0196298588157
FeaturesList : 0.0
FeaturesList : 0.161782432099
FeaturesList : 0.414186795229
FeaturesList : 0.113897443933
FeaturesList : 0.0146796769291
FeaturesList : 0.0850576242726
FeaturesList : 0.257385667092
FeaturesList : 0.196264537455
FeaturesList : 0.0129958411129
Feature

FeaturesList : 0.155691001426
FeaturesList : 0.153061895828
FeaturesList : 0.0720204972486
FeaturesList : 0.0369469405704
FeaturesList : 0.0634575153203
FeaturesList : 0.0183699562802
FeaturesList : 0.0
FeaturesList : 0.000422929658297
FeaturesList : 0.0485994126576
FeaturesList : 0.439571975648
FeaturesList : 0.162675534131
FeaturesList : 0.00814261351624
FeaturesList : 0.00705199307762
FeaturesList : 0.00989413500399
FeaturesList : 0.0
FeaturesList : 0.0
FeaturesList : 0.0148866151361
FeaturesList : 0.0624936135073
FeaturesList : 0.440525240066
FeaturesList : 0.0
FeaturesList : 0.0
FeaturesList : 0.0
FeaturesList : 0.00807444360914
FeaturesList : 0.206197536261
FeaturesList : 0.00350268882379
FeaturesList : 0.00351058722336
FeaturesList : 0.00660262446096
FeaturesList : 0.0399518379417
FeaturesList : 0.00840739964575
FeaturesList : 0.0205059615242
FeaturesList : 0.0159873956883
FeaturesList : 0.673944278418
FeaturesList : 0.00395006232271
FeaturesList : 0.00402777650986
FeaturesList 

FeaturesList : 0.0250032245174
FeaturesList : 0.546286303507
FeaturesList : 0.00939442405753
FeaturesList : 0.0074499407793
FeaturesList : 0.00599935061804
FeaturesList : 0.0955925646069
FeaturesList : 0.400334975583
FeaturesList : 0.212124162237
FeaturesList : 0.00550226971639
FeaturesList : 0.0210792858388
FeaturesList : 0.0245257361937
FeaturesList : 0.0109003187645
FeaturesList : 0.0
FeaturesList : 0.00139404131839
FeaturesList : 0.0269022177586
FeaturesList : 0.321321428406
FeaturesList : 0.00134090722999
FeaturesList : 0.0137328247608
FeaturesList : 0.00986747269941
FeaturesList : 0.082200348995
FeaturesList : 0.0132889148986
FeaturesList : 0.000524533159142
FeaturesList : 0.00344066730936
FeaturesList : 0.0253606754395
FeaturesList : 0.619614478899
FeaturesList : 0.0302545238715
FeaturesList : 0.0320782305182
FeaturesList : 0.0580939947884
FeaturesList : 0.151521723075
FeaturesList : 0.0
FeaturesList : 0.00551398090558
FeaturesList : 0.0264386482716
FeaturesList : 0.022610550124

FeaturesList : 0.15963365811
FeaturesList : 0.243219865695
FeaturesList : 0.0630428400637
FeaturesList : 0.0185756853419
FeaturesList : 0.0850704393449
FeaturesList : 0.195865882941
FeaturesList : 0.109179400092
FeaturesList : 0.130103473597
FeaturesList : 0.0687137986838
FeaturesList : 0.306847284368
FeaturesList : 0.145278750075
FeaturesList : 0.207461395333
FeaturesList : 0.0435118314171
FeaturesList : 0.156734804328
FeaturesList : 0.188682527111
FeaturesList : 0.15448759469
FeaturesList : 0.0517382933948
FeaturesList : 0.12084819672
FeaturesList : 0.233502835751
FeaturesList : 0.135933334059
FeaturesList : 0.053665454355
FeaturesList : 0.0581789947197
FeaturesList : 0.050491873885
FeaturesList : 0.364402777425
FeaturesList : 0.21511629797
FeaturesList : 0.0514727165208
FeaturesList : 0.0148837119896
FeaturesList : 0.172625533254
FeaturesList : 0.192924208599
FeaturesList : 0.10136042952
FeaturesList : 0.0900162477205
FeaturesList : 0.0498178667342
FeaturesList : 0.260979918734
Feat

In [5]:
import pickle
pickle.dump( FeaturesList, open( "FeaturesList.p", "wb" ) )
FeaturesList = pickle.load( open( "FeaturesList.p", "rb" ) )
# print FeaturesList

objImgClassifier=KNNClassifier(FeaturesList) 

print objImgClassifier.classify(FeaturesList[0][0])




C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in power
  


bedroom
